1. 代码参考：https://mimiyouyou.com/2021/11/03/%E5%BE%AE%E8%B0%83transformers%E8%BF%9B%E8%A1%8C%E5%9E%83%E5%9C%BE%E9%82%AE%E4%BB%B6%E5%88%86%E7%B1%BB/
2. 模型请自行去huggingface官网下。下载指导：https://zhuanlan.zhihu.com/p/147144376
3. 本模型需要gpu。

In [96]:
import pandas as pd
import numpy as np
train_data,test_data=pd.read_csv("train.csv").to_numpy(),pd.read_csv("test_noLabel.csv").to_numpy()
train_examples, train_labels=train_data[:,2],train_data[:,1]
test_examples=test_data[:,1]

train_labels[train_labels=="ham"]=0
train_labels[train_labels=="spam"]=1
print(train_labels)


train_labels = train_labels.astype(np.int64)

[0 1 0 ... 0 0 0]


In [97]:
from transformers import BertTokenizer

# 载入Bert tokenizer.
tokenizer = BertTokenizer.from_pretrained('./', do_lower_case=True)

In [98]:
sentences = train_data[:,2]
input_ids = []
attention_masks = []
 
for sent in sentences:
    # `encode_plus` 的步骤:
    #   (1) 分割文本；
    #   (2) 在文本开头加入 `[CLS]` 符号.
    #   (3) 在文本末尾加入 `[SEP]` 符号.
    #   (4) 把文本分割后的词序列映射为编号序列.
    #   (5) 把编号序列添加 `PAD` 符号扩展到 `max_length`.
    #   (6) 提供一个 `mask` 序列， 值为1表示对应的词不是扩充的，值为0表示对应的词是扩充的空白.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # 输入的句子.
                        add_special_tokens = True, # 添加 '[CLS]', '[SEP]'
                        max_length = 64,           # 扩展到的最大长度.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # 提供mask.
                        return_tensors = 'pt',     # 返回 pytorch tensors.
                   )
    
    # 把所有编号序列整合.    
    input_ids.append(encoded_dict['input_ids'])
    
    # 所有mask整合.
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/root/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [99]:
print(input_ids)
# input_ids=tuple(input_ids.tolist())

[tensor([[  101,  1045,  2123,  1005,  1056,  2031, 10334,  1005,  1055,  2193,
          1010,  1045,  2145,  4033,  1005,  1056,  2245,  2039,  1037, 11937,
          6593,  3993,  2126,  2000,  3198,  4074,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), tensor([[  101, 26478,  8609,  2015,   999,  1016,  4684,  1017,  2290,  2678,
         19093,  1054,  6737,  1012,  2655,  5641,  2692,  2575, 22022, 27814,
         17134,  2692,  2085,   999,  2678,  7507,  2102, 15536,  2094,  2115,
         14711,  1010,  2377,  9262,  2399,  1010, 21469, 10441,  2094, 26572,
          8458,  2189,  1010,  2053,  4179,  9278,  2140,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           

In [100]:
import torch
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = train_labels
labels = torch.tensor(labels)

In [101]:
from torch.utils.data import TensorDataset, random_split

# 封装进TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size

# 60%训练，40%验证
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [102]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 每轮训练输入的数据量，BERT作者推荐的值是16或32
batch_size = 32

# 两个DataLoaders，分别载入训练集和验证集.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # 随机选择batch训练
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), # 有序选择batch测试
            batch_size =batch_size
        )

In [103]:
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


In [104]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# 这是一个12层的模型，最后一层输出是标签
model = BertForSequenceClassification.from_pretrained(
    "./",
    num_labels = 2, # 二分类任务
    output_attentions = False, # 不返回 attentions weights.
    output_hidden_states = False, # 不返回所有的 hidden-states.
    
)

# 告诉PyTorch这个模型要在GPU上运行
model.cuda()

Some weights of the model checkpoint at ./ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./ and are new

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [105]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # 学习率，网络参数的更新系数
                  eps = 1e-8 # 如果新旧lr之间的差异小于eps，则忽略此次更新
                )

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [106]:
from transformers import get_linear_schedule_with_warmup

# 微调的训练轮次数，BERT作者建议是2~4轮
epochs = 8

#训练步骤数 [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# 学习率调整器
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [107]:
import numpy as np
import time
import datetime

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    # 取近似值
    elapsed_rounded = int(round((elapsed)))
    # 转化为时分秒
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [108]:
import random
import numpy as np

# 固定随机数种子，使得实验结果可重复
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 存储训练状态
training_stats = []

# 代码开始运行的时间
total_t0 = time.time()

# 每一轮训练：
for epoch_i in range(0, epochs):
    
    # ========================================
    #               训练过程
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 当前时间
    t0 = time.time()

    # 本轮训练loss
    total_train_loss = 0

    # 把BERT模型设置为训练状态
    model.train()

    # 每个Batch的数据：
    for step, batch in enumerate(train_dataloader):

        # 设置每40个batch输出一次log
        if step % 40 == 0 and not step == 0:
            # 时间消耗
            elapsed = format_time(time.time() - t0)
            
            # 训练进程
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 把当前batch的数据上传到GPU
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 清除之前的梯度，准备下一次训练
        model.zero_grad()        

        # 得到一个输出和loss
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        logits = output.logits

        # 累积loss
        total_train_loss += loss.item()

        # 前向传递参数
        loss.backward()

        # 梯度裁剪，防止梯度爆炸，将梯度约束在某一个区间之内，在训练的过程中，在优化器更新之前进行梯度截断操作
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 更新参数
        optimizer.step()

        # 更新学习率
        scheduler.step()

    # 计算所有batch的平均loss
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # 计算本轮训练花费的时间
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               验证
    # ========================================

    print("")
    print("Running Validation...")
    torch.cuda.empty_cache()
    
    t0 = time.time()
    # 把模型设置为验证模式
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # 逐batch验证
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # 设置模型无需不跟踪梯度，跟踪梯度是训练的时候要做的
        with torch.no_grad():        

            # logits是模型输出结果
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = output.loss
            logits = output.logits
            
        # 累计loss
        total_eval_loss += loss.item()

        # 从GPU转移到CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 累计准确率
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # 平均准确率
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.9f}".format(avg_val_accuracy))

    # 平均loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # 验证模型过程的时间消耗
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # 记录起来
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()


======== Epoch 1 / 8 ========
Training...
  Batch    40  of    138.    Elapsed: 0:00:03.
  Batch    80  of    138.    Elapsed: 0:00:06.
  Batch   120  of    138.    Elapsed: 0:00:09.

  Average training loss: 0.15
  Training epoch took: 0:00:10

Running Validation...
  Accuracy: 1.000000000
  Validation Loss: 0.01
  Validation took: 0:00:00

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    138.    Elapsed: 0:00:03.
  Batch    80  of    138.    Elapsed: 0:00:06.
  Batch   120  of    138.    Elapsed: 0:00:09.

  Average training loss: 0.03
  Training epoch took: 0:00:10

Running Validation...
  Accuracy: 1.000000000
  Validation Loss: 0.00
  Validation took: 0:00:00

======== Epoch 3 / 8 ========
Training...
  Batch    40  of    138.    Elapsed: 0:00:03.
  Batch    80  of    138.    Elapsed: 0:00:06.
  Batch   120  of    138.    Elapsed: 0:00:09.

  Average training loss: 0.02
  Training epoch took: 0:00:10

Running Validation...
  Accuracy: 1.000000000
  Validation Loss: 

In [109]:
sentences = test_data[:,1]
input_ids = []
attention_masks = []
 
for sent in sentences:
    # `encode_plus` 的步骤:
    #   (1) 分割文本；
    #   (2) 在文本开头加入 `[CLS]` 符号.
    #   (3) 在文本末尾加入 `[SEP]` 符号.
    #   (4) 把文本分割后的词序列映射为编号序列.
    #   (5) 把编号序列添加 `PAD` 符号扩展到 `max_length`.
    #   (6) 提供一个 `mask` 序列， 值为1表示对应的词不是扩充的，值为0表示对应的词是扩充的空白.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # 输入的句子.
                        add_special_tokens = True, # 添加 '[CLS]', '[SEP]'
                        max_length = 64,           # 扩展到的最大长度.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # 提供mask.
                        return_tensors = 'pt',     # 返回 pytorch tensors.
                   )
    
    # 把所有编号序列整合.    
    input_ids.append(encoded_dict['input_ids'])
    
    # 所有mask整合.
    attention_masks.append(encoded_dict['attention_mask'])

In [110]:
# 保存模型和分词器
model.save_pretrained('./my_model')
tokenizer.save_pretrained('./my_model')

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json')

In [111]:
test_data[:,1]

array(['Ok.ok ok..then..whats ur todays plan',
       '\\Hi darlin did youPhone me? Im atHome if youwanna chat.\\""',
       'K. Did you call me just now ah? ', ...,
       'You know there is. I shall speak to you in  &lt;#&gt;  minutes then',
       'How much u trying to get?',
       'We don call like  &lt;#&gt;  times oh. No give us hypertension oh.'],
      dtype=object)

In [112]:
# 设置GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cpu_device=torch.device('cpu')
# model.to(device)
model.to(cpu_device)
result=[]

for i in range(len(test_data)):
    # sentences是一个文本的list
    # sentences = test_data[i,1].flatten().tolist()
    sentences = [test_data[i,1]]
    # 分词
    sentences_encodings = tokenizer(sentences, truncation=True, padding=True)

    # 转换为张量
    sentences_inputs = torch.tensor(sentences_encodings['input_ids'])
    sentences_masks = torch.tensor(sentences_encodings['attention_mask'])

    # 上传到GPU
    sentences_inputs = sentences_inputs.to(cpu_device)
    sentences_masks = sentences_masks.to(cpu_device)

    # 模型预测
    outputs = model(sentences_inputs, attention_mask=sentences_masks)
    logits = outputs.logits

    # softmax操作
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # 预测结果
    preds = torch.argmax(probs, dim=-1).tolist()
    print(f"Predicted classes: {preds}")
    result.append(preds)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [1]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [1]
Predicted classes: [0]
Predicted classes: [1]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [1]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted classes: [0]
Predicted c

In [113]:
result=np.mean(result,axis=1)
result=list(map(int, result))
print(result)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 

In [114]:
res_labels=[]
for i in range(len(result)):
  if result[i]>0:
    res_labels.append("spam")
  else: 
    res_labels.append("ham")

print(res_labels)

['ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham

In [115]:
# 假设test和res都是numpy数组
# 把test的第一列和res合并成一个二维数组
output = np.column_stack((test_data[:, 0], res_labels))
# 使用np.savetxt()函数来输出为csv文件
np.savetxt('output.csv', output, fmt='%s', delimiter=',', header='ID,Label')
